In [1]:
#libraries used
import pandas as pd
import re
from datetime import datetime

In [34]:
#import of the data set
data = pd.read_csv('GSAF5.csv', encoding='latin-1')

In [3]:
#check the number of null values per column
null_cols = data.isnull().sum()
null_cols[null_cols > 0]
#Time have 3213 null values -> don't keep it because if this information is often missing, we won't be able to get a trend on the attack hour
#Species have 2934 null values -> keep it because we might still want to know what kind of shark attack
#Age have 2681 null values -> don't keep it as it is not an interresting information if often missing
#I wanted to remove Name column as I don't find this information useful but sometimes it contains sex and other information

Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
href formula                 1
href                         3
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

In [4]:
#function to clean the date from Case number attribute
def clean_Date(e):

    if re.search("[0-9]{4}.[0-9]{2}.[0-9]{2}",e):
        splitted_date = re.split("\D",e)
        if splitted_date[1] == "00":
            splitted_date[1] = "01"
        if splitted_date[2] == "00":
            splitted_date[2] = "01"
        cleaned_date = datetime(int(splitted_date[0]),int(splitted_date[1]),int(splitted_date[2]))
    else:
        cleaned_date = "Unknown Date"
    
    return cleaned_date

In [35]:
#function to change empty value by "Not specified"
def empty_values(e):
    if re.search("^$",e):
        e = "Not specified"
    return e

In [36]:
#function to remove space at the beginning and at the end of my string
def remove_spaces(e):
    if re.search("^ ",e):
        e = re.sub("^ +","",e)
    if re.search(" $",e):
        e = re.sub(" +$","",e)
    return e

In [20]:
def sex_cleaning(e):
    if ((e != 'F') or (e != 'M')):
        print(e)
        e = "Not specified" #retourne que du not specified
    else :
        e = 'A' 
    return e

In [8]:
def fatal_cleaning(e):
    if e != "N" or e != "Y":
        e = "Not specified"
    return e

In [9]:
#Clean Date column
#apply clean_date function to our Case number column and put the calculated date into date column
data['Date'] = data['Case Number'].apply(clean_Date)

In [10]:
#create a new column to know if the Date did not contain day and/or month
lst=[]
for e in data['Case Number']:
    if re.search("[0-9]{4}.0{2}.0{2}",e) or re.search("[0-9]{4}.[0-9]{2}.0{2}",e):
        lst.append("Yes")
    else:
        lst.append("No")
data['Date_alterated'] = pd.DataFrame(lst)

In [37]:
#rename our columns name by applying remove space function
data.rename(remove_spaces,axis=1, inplace=True)
data.columns.str.capitalize()

Index(['Case number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal (y/n)', 'Time',
       'Species', 'Investigator or source', 'Pdf', 'Href formula', 'Href',
       'Case number.1', 'Case number.2', 'Original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [12]:
#clean country
#Convert Country data into str because some are floats
#I remove "?", considering the country is correct
data['Country'] = data['Country'].astype(str)
data['Country'] = data['Country'].str.replace('\?', '')
data['Country'] = data['Country'].apply(remove_spaces)

In [13]:
#clean area
data['Area'] = data['Area'].astype(str)
data['Area'] = data['Area'].str.replace('\"','')
data['Area'] = data['Area'].str.replace("Tavenui","Taveuni")
data['Area'] = data['Area'].str.replace("Tamilnadu","Tamil Nadu")
data['Area'] = data['Area'].str.replace("Guerro","Guerrero")
data['Area'] = data['Area'].str.replace("Guerrrero","Guerrero")
data['Area'] = data['Area'].str.replace("islans","Islands")
data['Area'] = data['Area'].str.replace("Province","")
data['Area'] = data['Area'].str.replace("Region","")
data['Area'] = data['Area'].str.replace("shire","")
data['Area'] = data['Area'].apply(empty_values)
data['Area'] = data['Area'].apply(remove_spaces)

In [14]:
#Clean Location column
data['Location'] = data['Location'].astype(str)
data['Location'] = data['Location'].str.replace("\(","")
data['Location'] = data['Location'].str.replace("\)","")
data['Location'] = data['Location'].apply(empty_values)
data['Location'] = data['Location'].apply(remove_spaces)

In [15]:
#Clean Activity column
data['Activity'] = data['Activity'].astype(str)
#find verb in gerund
lst=[]
for e in data['Activity']:
    lst.append(re.findall("[A-Za-z]*-?ing",e))
data['Activity'] = pd.DataFrame(lst) #loss of information once there is more than one activity on a line

In [38]:
print(set(data['Sex']))

{nan, '.', 'N', 'M ', 'F', 'M', 'lli'}


In [16]:
for e in data['Sex']:
    print("type", type(e), "valeur", e)

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur F
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'float'> valeur nan
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <cl

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur F
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> v

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <cl

...

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'float'> valeur nan
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M


type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> v

type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur F
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'float'> valeur nan
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str'> valeur M
type <class 'str

In [41]:
#Clean Sex column
data['Sex'] = data['Sex'].astype(str)
data['Sex'] = data['Sex'].str.replace("N","")
data['Sex'] = data['Sex'].str.replace("\.","")
data['Sex'] = data['Sex'].str.replace("lli","")
data['Sex'] = data['Sex'].str.replace("nan","")

In [43]:
data['Sex'] = data['Sex'].apply(empty_values)
data['Sex'] = data['Sex'].apply(remove_spaces)


In [44]:
print(set(data['Sex']))

{'Not specified', 'M', 'F'}


In [21]:
data['Sex'] = data['Sex'].apply(sex_cleaning)

M
M
M
M
M
nan
M
F
M
F
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
nan
F
F
M
M
F
M
M
M
F
M
M
M
M
M
M
M
M
M
F
M
M
M
F
M
M
M
M
F
M
F
F
F
M
F
M
M
M
M
M
M
M
M
F
M
M
M
F
M
M
M
M
nan
M
M
M
M
M
M
M
F
M
M
F
M
M
M
F
M
M
M
M
M
M
M
F
M
M
M
M
nan
M
F
M
F
M
M
F
M
F
M
M
M
M
M
M
F
M
M
M
F
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
F
M
M
M
F
M
F
M
M
M
M
F
M
M
M
M
M
F
M
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
F
M
M
M
M
M
F
M
M
M
M
F
F
F
M
F
M
F
F
M
F
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
nan
F
M
M
M
M
M
M
M
F
M
M
M
F
M
M
nan
M
nan
nan
M
M
M
M
M
M
M
M
M
nan
M
M
M
F
M
F
M
F
M
F
F
F
F
M
M
M
M
M
M
M
M
F
M
M
M
F
M
M
F
M
nan
M
F
M
F
M
nan
M
M
M
M
F
M
M
F
M
M
M
M
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
F
F
M
F
F
M
M
M
F
M
F
M
M
F
M
M
M
M
nan
M
M
M
F
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
nan
M
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
F
F
M
M
M
M
M
M
M
M
F
M
F
M
M
F
F
F
M
M
M
F
M
M
F
M
M
M
F
F
M
F
M
F
M
M
M
M
M
M
M
M
M
M
F
F
M
M
M
M
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
nan
M
M
M
M
M
M


nan
M
M
M
M
M
M
M
M
M
nan
nan
nan
M
nan
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
nan
M
M
nan
F
M
M
M
M
nan
nan
M
M
M
M
M
M
nan
M
M
M
nan
M
M
nan
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
nan
M
M
M
M
M
M
nan
F
nan
nan
F
F
M
M
nan
nan
nan
nan
M
M
M
M
M
nan
M
M
M
M
M
M
M
F
nan
M
M
M
M
M
M
M
M
M
M
M
nan
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
nan
nan
M
M
M
M
M
nan
nan
M
M
M
M
M
M
M
nan
nan
M
M
M
M
M
nan
nan
M
M
M
M
M
nan
M
M
M
nan
M
nan
M
M
M
nan
M
M
M
nan
F
M
M
M
M
M
M
nan
M
M
M
M
M
M
M
M
M
F
M
M
nan
nan
M
M
M
F
M
M
M
M
M
M
M
M
M
M
nan
nan
M
F
M
M
M
F
M
M
M
M
F
M
F
M
N
M
M
M
M
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
F
M
M
M
F
M
nan
M
M
M
M
M
M
nan
M
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
M
M
F
M
M
M
F
M
M
M
M
nan
M
M
M
M
M
F
M
M
M
nan
M
M
M
M
M
M
M
M
M
M
nan
M
M
nan
M
M
M
M
M
M
nan
M
M
nan
M
M
M
nan
M
M
M
M
M
M
M
M
nan
M
M
M
M
M
nan
M
M
M
M
M
M
F
M
M
nan
M
M
M
M
M
M
F
M
M
M
M
M
M
M
M
M
M
M
M
nan
M
M
F
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
M
F
M
M


In [226]:
#I use Name column information to get the sex if precised
data['Name'] = data['Name'].astype(str)
data['Name'] = data['Name'].str.lower()
lst = []
for e in data['Name']:
    if re.search("boy",e) or re.search("male",e):
        lst.append("M")
    elif re.search("girl",e) or re.search("female",e):
        lst.append("F")
    else:
        lst.append("")
data['test sex'] = pd.DataFrame(lst)

In [46]:
#I use newly created column test sex information to complete sex column
data.loc[(data['Sex']=="Not specified") & (data['test sex']=="M"), 'Sex'] = "M"
data.loc[(data['Sex']=="Not specified") & data['test sex']=="F", 'Sex'] = "F"

C:\Users\Elodie\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [47]:
#Clean Injury column

In [23]:
#Clean Fatal (Y/N) column
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].astype(str)
data['Fatal (Y/N)'] = data['Fatal (Y/N)'].apply(remove_spaces)

data['Fatal (Y/N)'] = data['Fatal (Y/N)'].apply(fatal_cleaning)

In [24]:
print(set(data['Fatal (Y/N)']))

{'Not specified'}


In [228]:
data.to_csv('cleaned_data_shark.csv', index=False)